#Obtenido los datos de yahoo - precio de cierre contra 200 dma
#Acumulado contra el último precio de cierre


In [1]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

In [3]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')
ek.set_app_key(cfg['eikon']['app_id'])

At the start of every calendar month, we sell all holdings and buy 50 random stocks. 
The weight of each stock is still completely random, but they all add up to 100% 
to make sure we’re fully invested. 
The only added rule here is that if the index itself is below its 200-day moving average,
we don’t buy. 
This means that if the index is below the average on the first of the month, 
we spend the entire month in cash.
Now we do another 500 random iterations to see what comes out.

In [4]:
FTSE = ek.get_data('0#.FTSE', fields=['TR.CommonName', 'TR.PriceClose',
                                      'TR.Volume', 'TR.TotalReturnYTD'])[0]
FTSE

,Instrument,Company Common Name,Price Close,Volume,YTD Total Return
0,STAN.L,Standard Chartered PLC,660.0,23165678,0.808013
1,CRDA.L,Croda International PLC,4740.0,378252,-6.178218
2,ANTO.L,Antofagasta PLC,1825.0,904817,5.358738
3,BNZL.L,Bunzl plc,3074.0,393264,-2.978056
4,SGE.L,Sage Group PLC,1229.5,4926654,7.440737
...,...,...,...,...,...
95,EDV.L,Endeavour Mining plc,1471.0,360558,-14.633981
96,OCDO.L,Ocado Group PLC,456.8,2624690,-40.875527
97,LSEG.L,London Stock Exchange Group PLC,9318.0,2029241,-0.107828
98,TSCO.L,Tesco PLC,282.2,33775436,-1.480207


In [5]:
FTSC = ek.get_data('0#.FTSC', fields=['TR.CommonName', 'TR.PriceClose',
                                      'TR.Volume', 'TR.TotalReturnYTD'])[0]
FTSC

,Instrument,Company Common Name,Price Close,Volume,YTD Total Return
0,FSTA.L,Fuller Smith & Turner PLC,554.0,103157,-14.417178
1,IGC.L,India Capital Growth Fund Ltd,178.0,486932,5.202312
2,PINE.L,Pinewood Technologies Group PLC,37.6,707720,16.718266
3,OIT.L,Odyssean Investment Trust PLC,153.5,106904,-2.531646
4,NCYF.L,CQS New City High Yield Fund Ltd,50.8,1642709,2.387226
...,...,...,...,...,...
212,MNL.L,Manchester And London Investment Trust PLC,670.0,102070,25.411335
213,PORV.L,Porvair PLC,627.0,36419,1.298701
214,APTD.L,Aptitude Software Group PLC,332.0,52433,19.642857
215,PCA.L,Palace Capital PLC,224.0,67952,0.675676


In [6]:
import datetime as dt
today = dt.datetime.today()
f_inicio = today - dt.timedelta(days=600)
print(today, f_inicio)

2024-03-11 15:35:55.184325 2022-07-20 15:35:55.184325


In [7]:
df_ts = ek.get_timeseries(['.FTSE'], fields = ['CLOSE'],
                          start_date = f_inicio, 
                          end_date = today, 
                          interval="daily", 
                          corax = 'adjusted' )

df_ts

.FTSE,CLOSE
Date,
2022-07-21,7270.51
2022-07-22,7276.37
2022-07-25,7306.3
2022-07-26,7306.28
2022-07-27,7348.23
...,...
2024-03-05,7646.16
2024-03-06,7679.31
2024-03-07,7692.46


In [8]:
df_ts['SMA_200'] = df_ts['CLOSE'].rolling(200).mean()
filtered_df_ts = df_ts[~df_ts['SMA_200'].isna()]
filtered_df_ts

.FTSE,CLOSE,SMA_200
Date,,
2023-05-09,7764.09,7504.21320
2023-05-10,7741.33,7506.56730
2023-05-11,7730.58,7508.83835
2023-05-12,7754.62,7511.07995
2023-05-15,7777.7,7513.43705
...,...,...
2024-03-05,7646.16,7540.79540
2024-03-06,7679.31,7541.05645
2024-03-07,7692.46,7541.66440


In [9]:
# if the index itself is below its 200-day moving average, we don’t buy. 

filtered_df_ts['BUY'] = filtered_df_ts['SMA_200'] < filtered_df_ts['CLOSE']
filtered_df_ts

.FTSE,CLOSE,SMA_200,BUY
Date,,,
2023-05-09,7764.09,7504.21320,True
2023-05-10,7741.33,7506.56730,True
2023-05-11,7730.58,7508.83835,True
2023-05-12,7754.62,7511.07995,True
2023-05-15,7777.7,7513.43705,True
...,...,...,...
2024-03-05,7646.16,7540.79540,True
2024-03-06,7679.31,7541.05645,True
2024-03-07,7692.46,7541.66440,True


At the start of every calendar month, we sell all holdings and buy 50 random stocks. 
The weight of each stock is still completely random, but they all add up to 100% 


In [90]:
Inst_rndm = pd.DataFrame(FTSE['Instrument'].sample(n=50, random_state=140))
Inst_rndm['randNumCol'] = np.random.randint(1,10, size=len(Inst_rndm))
Inst_rndm['weight'] = Inst_rndm['randNumCol']/sum(Inst_rndm['randNumCol'] )
Inst_rndm

,Instrument,randNumCol,weight
64,MNDI.L,6,0.026906
9,AZN.L,1,0.004484
98,TSCO.L,8,0.035874
29,SKG.L,1,0.004484
60,PHNX.L,8,0.035874
39,UU.L,9,0.040359
1,CRDA.L,1,0.004484
43,PRU.L,2,0.008969
90,DPLM.L,5,0.022422
42,WTB.L,5,0.022422


In [21]:
sum(Inst_rndm['weight'])

1.0000000000000002

In [91]:
Inst_rndm['inversion'] = Inst_rndm['weight']*1000000
Inst_rndm

,Instrument,randNumCol,weight,inversion
64,MNDI.L,6,0.026906,26905.829596
9,AZN.L,1,0.004484,4484.304933
98,TSCO.L,8,0.035874,35874.439462
29,SKG.L,1,0.004484,4484.304933
60,PHNX.L,8,0.035874,35874.439462
39,UU.L,9,0.040359,40358.744395
1,CRDA.L,1,0.004484,4484.304933
43,PRU.L,2,0.008969,8968.609865
90,DPLM.L,5,0.022422,22421.524664
42,WTB.L,5,0.022422,22421.524664


In [101]:

df_ts_rndm = ek.get_timeseries(['MNDI.L'], fields = ['CLOSE'],
                          start_date = dt.datetime(2024, 2, 1, 0, 0), 
                          end_date = dt.datetime(2024, 2, 29, 0, 0), 
                          interval="daily", 
                          corax = 'adjusted' )

Precios = df_ts_rndm = df_ts_rndm.rename(columns={'CLOSE': 'MNDI.L'})

SyntaxError: invalid syntax (3206118801.py, line 1)

In [102]:
Precios 

MNDI.L,MNDI.L
Date,
2024-02-01,1401.0
2024-02-02,1367.5
2024-02-05,1364.0
2024-02-06,1373.0
2024-02-07,1381.0
2024-02-08,1336.0
2024-02-09,1344.0
2024-02-12,1360.5
2024-02-13,1346.0


In [103]:
for i in Inst_rndm['Instrument'][1:]:
    print(i)
    w = ek.get_timeseries([i], fields = ['CLOSE'],
                          start_date = dt.datetime(2024, 2, 1, 0, 0), 
                          end_date = dt.datetime(2024, 2, 29, 0, 0), 
                          interval="daily", 
                          corax = 'adjusted' )
    w1 = w.rename(columns={'CLOSE': i})
    Precios = Precios.join(w1, lsuffix="_left", rsuffix="_right")

AZN.L
TSCO.L
SKG.L
PHNX.L
UU.L
CRDA.L
PRU.L
DPLM.L
WTB.L
ANTO.L
WPP.L
ICP.L
FRAS.L
SGE.L
WEIR.L
EDV.L
BMEB.L
AAL.L
RR.L
LSEG.L
ULVR.L
DGE.L
BATS.L
ICAG.L
BP.L
LAND.L
REL.L
NG.L
RS1R.L
SMIN.L
SMDS.L
AAF.L
NWG.L
MNG.L
FLTRF.L
CTEC.L
EXPN.L
RIO.L
HLN.L
AHT.L
FCIT.L
BAES.L
SGRO.L
IMB.L
SSE.L
BDEV.L
BNZL.L
ITRK.L
LLOY.L


In [104]:
Precios

,MNDI.L,AZN.L,TSCO.L,SKG.L,PHNX.L,UU.L,CRDA.L,PRU.L,DPLM.L,WTB.L,...,AHT.L,FCIT.L,BAES.L,SGRO.L,IMB.L,SSE.L,BDEV.L,BNZL.L,ITRK.L,LLOY.L
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-01,1401.0,10430,282.9,2880,502.0,1062.5,4967,806.2,3340,3541,...,5248,965,1169.5,866.6,1880.0,1670.5,534.8,3204,4481,42.33
2024-02-02,1367.5,10360,290.8,2862,503.4,1057.0,4863,814.2,3320,3515,...,5344,979,1172.0,862.0,1871.0,1668.5,529.4,3208,4465,42.315
2024-02-05,1364.0,10360,289.8,2862,497.2,1049.5,4943,817.8,3258,3520,...,5112,980,1179.0,856.4,1882.0,1679.0,520.8,3203,4455,41.89
2024-02-06,1373.0,10534,289.9,2870,502.0,1049.0,4959,849.2,3294,3543,...,5240,990,1205.5,860.4,1892.5,1669.5,530.0,3212,4477,41.88
2024-02-07,1381.0,10490,280.0,2972,497.1,1044.5,4990,830.2,3290,3539,...,5244,987,1198.5,845.0,1868.0,1653.0,501.0,3189,4388,41.58
2024-02-08,1336.0,9823,280.4,3130,493.4,1025.5,5022,818.2,3386,3467,...,5264,984,1195.0,846.6,1858.5,1575.5,487.5,3200,4434,41.515
2024-02-09,1344.0,9761,280.5,3176,489.2,1016.5,4970,798.0,3380,3482,...,5250,985,1209.5,835.0,1882.5,1557.0,487.1,3202,4467,41.4
2024-02-12,1360.5,9501,279.6,3202,496.7,1032.0,4927,812.4,3386,3507,...,5278,995,1224.0,837.4,1896.0,1564.0,488.5,3198,4416,41.4
2024-02-13,1346.0,9600,273.7,3156,488.8,1026.5,4834,790.8,3300,3446,...,5116,976,1212.5,813.8,1883.5,1547.0,466.1,3179,4323,41.185


In [105]:
import math
Inst_rndm['Precios']  = Precios.iloc[0].tolist()
Inst_rndm['Activos'] = (Inst_rndm['inversion']/Inst_rndm['Precios']).apply(np.floor)
Inst_rndm['InversionInicial'] = Inst_rndm['Activos']*Inst_rndm['Precios']
Inst_rndm

,Instrument,randNumCol,weight,inversion,Precios,Activos,InversionInicial
64,MNDI.L,6,0.026906,26905.829596,1401.00,19.0,26619.00
9,AZN.L,1,0.004484,4484.304933,10430.00,0.0,0.00
98,TSCO.L,8,0.035874,35874.439462,282.90,126.0,35645.40
29,SKG.L,1,0.004484,4484.304933,2880.00,1.0,2880.00
60,PHNX.L,8,0.035874,35874.439462,502.00,71.0,35642.00
39,UU.L,9,0.040359,40358.744395,1062.50,37.0,39312.50
1,CRDA.L,1,0.004484,4484.304933,4967.00,0.0,0.00
43,PRU.L,2,0.008969,8968.609865,806.20,11.0,8868.20
90,DPLM.L,5,0.022422,22421.524664,3340.00,6.0,20040.00
42,WTB.L,5,0.022422,22421.524664,3541.00,6.0,21246.00


In [106]:
import math
Inst_rndm['PreciosFinal']  = Precios.iloc[-1].tolist()
Inst_rndm['InversionFinal']  = Inst_rndm['Activos']*Inst_rndm['PreciosFinal']
Inst_rndm

,Instrument,randNumCol,weight,inversion,Precios,Activos,InversionInicial,PreciosFinal,InversionFinal
64,MNDI.L,6,0.026906,26905.829596,1401.00,19.0,26619.00,1408.50,26761.50
9,AZN.L,1,0.004484,4484.304933,10430.00,0.0,0.00,9983.00,0.00
98,TSCO.L,8,0.035874,35874.439462,282.90,126.0,35645.40,278.80,35128.80
29,SKG.L,1,0.004484,4484.304933,2880.00,1.0,2880.00,3378.00,3378.00
60,PHNX.L,8,0.035874,35874.439462,502.00,71.0,35642.00,497.30,35308.30
39,UU.L,9,0.040359,40358.744395,1062.50,37.0,39312.50,1025.00,37925.00
1,CRDA.L,1,0.004484,4484.304933,4967.00,0.0,0.00,4766.00,0.00
43,PRU.L,2,0.008969,8968.609865,806.20,11.0,8868.20,777.60,8553.60
90,DPLM.L,5,0.022422,22421.524664,3340.00,6.0,20040.00,3456.00,20736.00
42,WTB.L,5,0.022422,22421.524664,3541.00,6.0,21246.00,3303.00,19818.00


In [107]:
print("Inversión Inicial:", sum(Inst_rndm['InversionInicial']))
print("Inversión Final:", sum(Inst_rndm['InversionFinal']))

Inversión Inicial: 945154.68
Inversión Final: 950338.52


In [108]:
v0 = df_ts.loc['20240201'][0]


In [109]:
v1 = df_ts.loc['20240229'][0]

In [110]:
print("Inversion final si hubiera invertido en el índice ", (945154.68*v1)/v0)

Inversion final si hubiera invertido en el índice  946129.3270534337


In [89]:
filtered_df_ts.loc['20240201']

.FTSE
CLOSE        7622.16
SMA_200    7558.9437
BUY             True
Name: 2024-02-01 00:00:00, dtype: object

At the start of every calendar month, we sell all holdings and buy 50 random stocks. The weight of each stock is still completely random, but they all add up to 100% to make sure we’re fully invested. The only added rule here is that if the index itself is below its 200-day moving average, we don’t buy. This means that if the index is below the average on the first of the month, we spend the entire month in cash. Now we do another 500 random iterations to see what comes out.

In [ ]:
Noviembre falla

In [111]:
Precios

,MNDI.L,AZN.L,TSCO.L,SKG.L,PHNX.L,UU.L,CRDA.L,PRU.L,DPLM.L,WTB.L,...,AHT.L,FCIT.L,BAES.L,SGRO.L,IMB.L,SSE.L,BDEV.L,BNZL.L,ITRK.L,LLOY.L
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-01,1401.0,10430,282.9,2880,502.0,1062.5,4967,806.2,3340,3541,...,5248,965,1169.5,866.6,1880.0,1670.5,534.8,3204,4481,42.33
2024-02-02,1367.5,10360,290.8,2862,503.4,1057.0,4863,814.2,3320,3515,...,5344,979,1172.0,862.0,1871.0,1668.5,529.4,3208,4465,42.315
2024-02-05,1364.0,10360,289.8,2862,497.2,1049.5,4943,817.8,3258,3520,...,5112,980,1179.0,856.4,1882.0,1679.0,520.8,3203,4455,41.89
2024-02-06,1373.0,10534,289.9,2870,502.0,1049.0,4959,849.2,3294,3543,...,5240,990,1205.5,860.4,1892.5,1669.5,530.0,3212,4477,41.88
2024-02-07,1381.0,10490,280.0,2972,497.1,1044.5,4990,830.2,3290,3539,...,5244,987,1198.5,845.0,1868.0,1653.0,501.0,3189,4388,41.58
2024-02-08,1336.0,9823,280.4,3130,493.4,1025.5,5022,818.2,3386,3467,...,5264,984,1195.0,846.6,1858.5,1575.5,487.5,3200,4434,41.515
2024-02-09,1344.0,9761,280.5,3176,489.2,1016.5,4970,798.0,3380,3482,...,5250,985,1209.5,835.0,1882.5,1557.0,487.1,3202,4467,41.4
2024-02-12,1360.5,9501,279.6,3202,496.7,1032.0,4927,812.4,3386,3507,...,5278,995,1224.0,837.4,1896.0,1564.0,488.5,3198,4416,41.4
2024-02-13,1346.0,9600,273.7,3156,488.8,1026.5,4834,790.8,3300,3446,...,5116,976,1212.5,813.8,1883.5,1547.0,466.1,3179,4323,41.185


In [120]:
Inst_rndm['Instrument2'] = Inst_rndm['Instrument'] + '_inv' 
Activos = Inst_rndm[['Instrument2', 'Activos']].set_index('Instrument2').transpose()

In [123]:
Precios['tmp'] = 1
Activos['tmp'] = 1
df = pd.merge(Precios, Activos, on=['tmp'])
df = df.drop('tmp', axis=1)

In [125]:
df.columns

Index(['MNDI.L', 'AZN.L', 'TSCO.L', 'SKG.L', 'PHNX.L', 'UU.L', 'CRDA.L',
       'PRU.L', 'DPLM.L', 'WTB.L', 'ANTO.L', 'WPP.L', 'ICP.L', 'FRAS.L',
       'SGE.L', 'WEIR.L', 'EDV.L', 'BMEB.L', 'AAL.L', 'RR.L', 'LSEG.L',
       'ULVR.L', 'DGE.L', 'BATS.L', 'ICAG.L', 'BP.L', 'LAND.L', 'REL.L',
       'NG.L', 'RS1R.L', 'SMIN.L', 'SMDS.L', 'AAF.L', 'NWG.L', 'MNG.L',
       'FLTRF.L', 'CTEC.L', 'EXPN.L', 'RIO.L', 'HLN.L', 'AHT.L', 'FCIT.L',
       'BAES.L', 'SGRO.L', 'IMB.L', 'SSE.L', 'BDEV.L', 'BNZL.L', 'ITRK.L',
       'LLOY.L', 'MNDI.L_inv', 'AZN.L_inv', 'TSCO.L_inv', 'SKG.L_inv',
       'PHNX.L_inv', 'UU.L_inv', 'CRDA.L_inv', 'PRU.L_inv', 'DPLM.L_inv',
       'WTB.L_inv', 'ANTO.L_inv', 'WPP.L_inv', 'ICP.L_inv', 'FRAS.L_inv',
       'SGE.L_inv', 'WEIR.L_inv', 'EDV.L_inv', 'BMEB.L_inv', 'AAL.L_inv',
       'RR.L_inv', 'LSEG.L_inv', 'ULVR.L_inv', 'DGE.L_inv', 'BATS.L_inv',
       'ICAG.L_inv', 'BP.L_inv', 'LAND.L_inv', 'REL.L_inv', 'NG.L_inv',
       'RS1R.L_inv', 'SMIN.L_inv', 'SMDS.L_inv',

In [127]:
df['Valor'] = df['MNDI.L']*df['MNDI.L_inv'] + df['AZN.L']*df['AZN.L_inv']+ df['TSCO.L']*df['TSCO.L_inv']+df['SKG.L']*df['SKG.L_inv']+ df['PHNX.L']*df['PHNX.L_inv']+df['UU.L']*df['UU.L_inv']+ df['CRDA.L']*df['CRDA.L_inv']+ df['PRU.L']*df['PRU.L_inv']+ df['DPLM.L']*df['DPLM.L_inv']+ df['WTB.L']*df['WTB.L_inv']+ df['ANTO.L']*df['ANTO.L_inv']+df['WPP.L']*df['WPP.L_inv']+ df['ICP.L']*df['ICP.L_inv']+df['FRAS.L']*df['FRAS.L_inv']+ df['SGE.L']*df['SGE.L_inv']+ df['WEIR.L']*df['WEIR.L_inv']
#'EDV.L''BMEB.L''AAL.L''RR.L''LSEG.L',
#       'ULVR.L''DGE.L''BATS.L''ICAG.L''BP.L''LAND.L''REL.L',
#       'NG.L''RS1R.L''SMIN.L''SMDS.L''AAF.L''NWG.L''MNG.L',
#       'FLTRF.L''CTEC.L''EXPN.L''RIO.L''HLN.L''AHT.L''FCIT.L',
#       'BAES.L''SGRO.L''IMB.L''SSE.L''BDEV.L''BNZL.L''ITRK.L',
#       'LLOY.L'

In [128]:
df

,MNDI.L,AZN.L,TSCO.L,SKG.L,PHNX.L,UU.L,CRDA.L,PRU.L,DPLM.L,WTB.L,...,FCIT.L_inv,BAES.L_inv,SGRO.L_inv,IMB.L_inv,SSE.L_inv,BDEV.L_inv,BNZL.L_inv,ITRK.L_inv,LLOY.L_inv,Valor
0,1401.0,10430,282.9,2880,502.0,1062.5,4967,806.2,3340,3541,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,311441.6
1,1367.5,10360,290.8,2862,503.4,1057.0,4863,814.2,3320,3515,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,311497.8
2,1364.0,10360,289.8,2862,497.2,1049.5,4943,817.8,3258,3520,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,308693.8
3,1373.0,10534,289.9,2870,502.0,1049.0,4959,849.2,3294,3543,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,312458.9
4,1381.0,10490,280.0,2972,497.1,1044.5,4990,830.2,3290,3539,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,309346.1
5,1336.0,9823,280.4,3130,493.4,1025.5,5022,818.2,3386,3467,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,306845.4
6,1344.0,9761,280.5,3176,489.2,1016.5,4970,798.0,3380,3482,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,306063.4
7,1360.5,9501,279.6,3202,496.7,1032.0,4927,812.4,3386,3507,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,308783.7
8,1346.0,9600,273.7,3156,488.8,1026.5,4834,790.8,3300,3446,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,303627.4
9,1352.5,9724,277.2,3182,495.4,1023.0,4865,790.4,3310,3445,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,305897.9


In [129]:

df_ts_rndm = ek.get_timeseries(['MNDI.L'], fields = ['CLOSE'],
                          start_date = dt.datetime(2022, 1, 1, 0, 0), 
                          end_date = dt.datetime(2024, 3, 11, 0, 0), 
                          interval="daily", 
                          corax = 'adjusted' )

Precios2 = df_ts_rndm = df_ts_rndm.rename(columns={'CLOSE': 'MNDI.L'})

In [130]:
for i in Inst_rndm['Instrument'][1:]:
    print(i)
    w = ek.get_timeseries([i], fields = ['CLOSE'],
                          start_date = dt.datetime(2022, 1, 1, 0, 0), 
                          end_date = dt.datetime(2024, 3, 11, 0, 0), 
                          interval="daily", 
                          corax = 'adjusted' )
    w1 = w.rename(columns={'CLOSE': i})
    Precios2 = Precios2.join(w1, lsuffix="_left", rsuffix="_right")

AZN.L
TSCO.L
SKG.L
PHNX.L
UU.L
CRDA.L
PRU.L
DPLM.L
WTB.L
ANTO.L
WPP.L
ICP.L
FRAS.L
SGE.L
WEIR.L
EDV.L
BMEB.L
AAL.L
RR.L
LSEG.L
ULVR.L
DGE.L
BATS.L
ICAG.L
BP.L
LAND.L
REL.L
NG.L
RS1R.L
SMIN.L
SMDS.L
AAF.L
NWG.L
MNG.L
FLTRF.L
CTEC.L
EXPN.L
RIO.L
HLN.L
AHT.L
FCIT.L
BAES.L
SGRO.L
IMB.L
SSE.L
BDEV.L
BNZL.L
ITRK.L
LLOY.L


In [131]:
Precios2

,MNDI.L,AZN.L,TSCO.L,SKG.L,PHNX.L,UU.L,CRDA.L,PRU.L,DPLM.L,WTB.L,...,AHT.L,FCIT.L,BAES.L,SGRO.L,IMB.L,SSE.L,BDEV.L,BNZL.L,ITRK.L,LLOY.L
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,1852.913185,8473,293.95,4112,662.4,1091.5,9906,1315.0,3400,3134,...,6090,946,554.6,1383.5,1644.5,1664.0,760.0,2867,5782,49.91
2022-01-05,1867.904392,8460,296.7,4168,659.8,1083.0,9808,1294.5,3384,3156,...,6200,940,547.0,1385.5,1648.5,1643.5,757.6,2871,5700,50.68
2022-01-06,1853.412892,8356,292.3,4140,659.0,1072.5,9504,1285.5,3254,3169,...,6114,925,549.8,1354.5,1638.5,1625.0,739.4,2777,5640,52.0
2022-01-07,1887.892668,8399,289.05,4139,670.0,1068.0,9458,1320.0,3208,3165,...,6058,913,558.6,1343.0,1644.0,1634.0,735.4,2765,5616,52.84
2022-01-10,1867.904392,8334,292.45,4078,669.6,1055.0,9034,1296.0,3054,3162,...,5700,893,572.0,1312.0,1664.0,1638.5,699.6,2696,5496,52.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-05,1385.0,10112,276.3,3338,495.9,1023.0,4671,757.2,3382,3172,...,5192,978,1280.0,867.8,1666.0,1658.5,471.0,3087,4905,47.8
2024-03-06,1384.0,10140,280.4,3254,499.8,1033.0,4683,759.2,3416,3204,...,5182,985,1276.0,873.0,1678.5,1668.5,472.5,3065,4959,48.4
2024-03-07,1380.0,10234,283.7,3296,507.8,1054.0,4816,781.2,3412,3224,...,5126,983,1272.0,880.4,1692.5,1661.5,477.0,3068,4960,48.605


In [132]:
Precios2['tmp'] = 1
Activos['tmp'] = 1
dfn = pd.merge(Precios2, Activos, on=['tmp'])
dfn = dfn.drop('tmp', axis=1)

In [133]:
dfn['Valor'] = dfn['MNDI.L']*dfn['MNDI.L_inv'] + dfn['AZN.L']*dfn['AZN.L_inv']+ dfn['TSCO.L']*dfn['TSCO.L_inv']+dfn['SKG.L']*dfn['SKG.L_inv']+ dfn['PHNX.L']*dfn['PHNX.L_inv']+dfn['UU.L']*dfn['UU.L_inv']+ dfn['CRDA.L']*dfn['CRDA.L_inv']+ dfn['PRU.L']*dfn['PRU.L_inv']+ dfn['DPLM.L']*dfn['DPLM.L_inv']+ dfn['WTB.L']*dfn['WTB.L_inv']+ dfn['ANTO.L']*dfn['ANTO.L_inv']+dfn['WPP.L']*dfn['WPP.L_inv']+ dfn['ICP.L']*dfn['ICP.L_inv']+dfn['FRAS.L']*dfn['FRAS.L_inv']+ dfn['SGE.L']*dfn['SGE.L_inv']+ dfn['WEIR.L']*dfn['WEIR.L_inv']
#'EDV.L''BMEB.L''AAL.L''RR.L''LSEG.L',
#       'ULVR.L''DGE.L''BATS.L''ICAG.L''BP.L''LAND.L''REL.L',
#       'NG.L''RS1R.L''SMIN.L''SMDS.L''AAF.L''NWG.L''MNG.L',
#       'FLTRF.L''CTEC.L''EXPN.L''RIO.L''HLN.L''AHT.L''FCIT.L',
#       'BAES.L''SGRO.L''IMB.L''SSE.L''BDEV.L''BNZL.L''ITRK.L',
#       'LLOY.L'

In [134]:
dfn

,MNDI.L,AZN.L,TSCO.L,SKG.L,PHNX.L,UU.L,CRDA.L,PRU.L,DPLM.L,WTB.L,...,FCIT.L_inv,BAES.L_inv,SGRO.L_inv,IMB.L_inv,SSE.L_inv,BDEV.L_inv,BNZL.L_inv,ITRK.L_inv,LLOY.L_inv,Valor
0,1852.913185,8473,293.95,4112,662.4,1091.5,9906,1315.0,3400,3134,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,346440.150519
1,1867.904392,8460,296.7,4168,659.8,1083.0,9808,1294.5,3384,3156,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,347524.083452
2,1853.412892,8356,292.3,4140,659.0,1072.5,9504,1285.5,3254,3169,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,343129.94495
3,1887.892668,8399,289.05,4139,670.0,1068.0,9458,1320.0,3208,3165,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,344825.560696
4,1867.904392,8334,292.45,4078,669.6,1055.0,9034,1296.0,3054,3162,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,342255.183452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,1385.0,10112,276.3,3338,495.9,1023.0,4671,757.2,3382,3172,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,306357.4
547,1384.0,10140,280.4,3254,499.8,1033.0,4683,759.2,3416,3204,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,308191.2
548,1380.0,10234,283.7,3296,507.8,1054.0,4816,781.2,3412,3224,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,311072.9
549,1348.5,10196,286.2,3336,510.0,1057.0,4738,779.0,3418,3226,...,9.0,23.0,31.0,19.0,21.0,75.0,1.0,1.0,741.0,310374.0


In [136]:
dfn['Valor'].std()

17666.622167075333

In [137]:
dfn['Valor'].mean()

303365.6996502178

In [ ]:
PyQuant News 🐍
@pyquantnews
·
Feb 19
If someone tells you they make money with technical analysis, ask them if their strategy is:

• Statistically significant?
• Consistently making money?
• Robust to parameter changes?

Be careful with technical analysis.